# LIME and SHAP Hands-on exercise
Welcome to the hands-on exercise for the DMTS Master Class on Explanable AI using LIME and SHAP frameworks.

This assignment uses Stroke Prediction Dataset that is used to predict whether a patient is likely to get stroke based on the input parameters like gender, age, various diseases, and smoking status or not. The dataset is available at https://www.kaggle.com/datasets/fedesoriano/stroke-prediction-dataset

Attribute Information
1. id: unique identifier
2. gender: "Male", "Female" or "Other"
3. age: age of the patient
4. hypertension: 0 if the patient doesn't have hypertension, 1 if the patient has hypertension
5. heart_disease: 0 if the patient doesn't have any heart diseases, 1 if the patient has a heart disease
6. ever_married: "No" or "Yes"
7. work_type: "children", "Govt_jov", "Never_worked", "Private" or "Self-employed"
8. Residence_type: "Rural" or "Urban"
9. avg_glucose_level: average glucose level in blood
10. bmi: body mass index
11. smoking_status: "formerly smoked", "never smoked", "smokes" or "Unknown"*
12. stroke: This is the output, which indicates if the person has Stroke or not. 1 if the patient had a stroke or 0 if not

*Note: "Unknown" in smoking_status means that the information is unavailable for this patient

The first few steps transforms the dataset into a form that is more efficient for Machine Learning. Then a Random Forest Classifier model is trained and validated. The result of the predictions are then analyzed using both LIME and SHAP libraries.


### 1 - Import Packages
First import all the packages that you will need during this assignment.

* pandas is the Python-based data analysis toolkit
* numpy is the fundamental package for scientific computing with Python
* matplotlib is a library for plotting graphs in Python
* seaborn is a data visualization library built on top of Matplotlib
* interpret (InterpretML) is an open-source Python package which exposes machine learning interpretability algorithms
* imblearn is a library to generate data to overcome imbalanced dataset

If you are using Google colab, it is likely that interpret and imblearn are not pre-installed. The below steps first attempts to import interpret and imblearn. If it is not found, it attempts to install. 
You might receive Warning on incompatability of few libraries. We can ignore it, as it does not impact this exercise. The runtime has to be restarted either by clicking on 'Restart Runtime' that is displayed in the out of the cell or from top menu select “Runtime” -> Restart Runtime

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

#!pip install interpret if not found
try:
 import interpret
except:
  print("Installing interpret ML")
  !pip install interpret


try:
  import imblearn
except:
  print("Installing imblearn")
  !pip install imblearn

### 2. Read the dataset
The below step loads the dataset which is a copy from Kaggle (https://www.kaggle.com/datasets/fedesoriano/stroke-prediction-dataset)
Once loaded the first few records are printed with the column names. Please observe the features in the form of columns and the values.

Please note: If you get the error "name 'pd' is not defined" please rerun the above cell and continue.

In [ ]:
df = pd.read_csv('https://raw.githubusercontent.com/dmts-techtonic/masterclass-xai/main/healthcare-dataset-stroke-data.csv')
df.head()

### 3. Analyze the data type
The info() method on the dataframe prints the data type of each of the attributes

In [ ]:
df.info()

### 4. Converting Categorical features
As you would have observed the features 'gender', 'ever_married', 'work_type', 'Residence_type', 'smoking_status' are non-numerical. Machine learning algorithms are good at dealing with numeric values. There are different ways to convert Categorical values to numerical. One-hot encoding technique is used below, where the each categorical value is converted into a new column (also called dummy column) and assign a binary value of 1 or 0 to those columns. Once converted the original non-numerical column is dropped. Observe the converted dataset

In [ ]:
# One-hot encode all categorical columns
categorical_cols = ["gender",
                        "ever_married",
                        "work_type",
                        "Residence_type",
                        "smoking_status"]

encoded = pd.get_dummies(df[categorical_cols], 
                        prefix=categorical_cols)

# Update data with new columns
df = pd.concat([encoded, df], axis=1)
df.drop(categorical_cols, axis=1, inplace=True)

# Impute missing values of BMI
df.bmi = df.bmi.fillna(0)

# Drop id as it is not relevant
df.drop(["id"], axis=1, inplace=True)

df.head()

### 5. Split Dataset for Training and Validation
The dataset is split between Training and Validation/Test in the ratio of 80:20. To keep the exercise simple, the original dataset is split into 2 instead of 3 separate sets for Training, Validation and Test. Hence the validation and test datasets are the same. 

The label column 'stroke' is extracted into result Dataframe and removed from both the training and validation data. 
Observe the total number of number of records in the Training and Validation.

In [ ]:
from sklearn.model_selection import train_test_split

X = df.drop('stroke',axis=1)
y = df['stroke']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=30)

print("Training Data : {} Features, {} Records".format(X_train.shape[1], X_train.shape[0]))
print("Validation Data : {} Features, {} Records".format(X_test.shape[1], X_test.shape[0]))

### 6. Feature scaling
Several Machine learning algorithms work efficiently if values of all the features are comparable and within similar range. In the below step MinMaxScaler is applied to normalize the feature values to the training and validation data.
In this exercise, we are going to skip the Feature scaling, and retain the original feature value so that during explainability it will be easy to relate to the original feature value. 

In [ ]:
'''
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)
'''

### 7. View imbalance of Label data
The below step displays the number of occurrences when the patient had stroke vs no stroke in the given dataset. Please observe the count when stroke is 0 and when stroke is 1. The number of records with stroke is lot less when compared to no stroke. 

In [ ]:
pd.DataFrame(y_train).value_counts()

### 8. Balance the data using Oversampling
As seen above, the training with such skewed test data may not be optimal. In order to balance the data Random Over Sampling technique is used. The RandomOverSampler duplicates some of the original samples having stroke '1'. Observe the result after oversampling.
The dataset is ready for training now!

In [ ]:
from imblearn.over_sampling import RandomOverSampler

oversample = RandomOverSampler(sampling_strategy='minority')
X_train, y_train = oversample.fit_resample(X_train, y_train)

pd.DataFrame(y_train).value_counts()

### 9. Model Creation and Training
The below steps creates Random Forest Classifier model, trains the model using the training data and then using the trained model, predicts the result for the validation data.

Random forests Classification algorithm is used in the exercise as it is a good representative "blackbox" model and also gives resonable good model performance. Any other algorithm also can be used for LIME and SHAP analysis.

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score, accuracy_score
from sklearn.metrics import classification_report,confusion_matrix
# %% Fit blackbox model
rf = RandomForestClassifier()
rf.fit(X_train.values, y_train.values)
y_pred = rf.predict(X_test.values)


### 10. Model Performance
Congratulations! with the completion of the above step, you have successfully trained a Machine Learning Model. 
Let us see the performance of the model by comparing the predicted values against the actual result of the validation/test data. The Accuracy and F1 score of the model is printed, along with the Confusion matrix that was used to calculate them.

You may be getting accuracy of 95% and F1 score of 0.54.  To keep the exercise simple, no hyperparameters tuning is done.  


In [ ]:
print(f"F1 Score {f1_score(y_test, y_pred, average='macro')}")
print(f"Accuracy {accuracy_score(y_test, y_pred)}")

# print(classification_report(y_test,y_pred))
confusion_matrix(y_test,y_pred)

### 11. Feature Importance
Now that the model has been trained and validated, let us see the importance of each of the features to understand how the model works. As it can be seen age, avg_glucose_level, bmi have high importance when compared to other features. While the feature importance gives the overall influence of the features, in order to get more insight on the influence of features for specific sample, LIME and SHAP can be used that are covered in the next steps. 

In [ ]:
feat_importances = pd.Series(rf.feature_importances_, index=X_train.columns)
feat_importances.nlargest(25).plot(kind='barh',figsize=(10,10))

# LIME - Local Interpretable Model Agnostic Explanations
The below few steps use LIME to explain the model prediction for Random Forest model created in the earlier step. As discussed in the theory session LIME is used to explain complex non-linear models with simple surrogate models. LIME is model agnostic and can be used for model created using any algorithm beyond Random Forest used in this exercise. LIME explains the classifier for a specific single instance and is therefore suitable for local consideration.
The research  paper “Why Should I Trust You? Explaining the Predictions of Any Classifier" by Marco Tulio Ribeiro, Sameer Singh, Carlos Guestrin describes in detail the mathematical modeling and internal working of LIME to explain the prediction. Fortunately the LIME libraries that we use below abstract the mathematical modeling and simplifies the usage.

### 12. Initialize LIME for Tabular data using InterpretML
The below step initializes LIME for Tabular data analysis. It uses interpretML library


In [ ]:
from interpret.blackbox import LimeTabular
from interpret import show

# %% Apply lime
# Initilize Lime for Tabular data
lime = LimeTabular(predict_fn=rf.predict_proba, 
                   data=X_train,
                   feature_names=df.drop('stroke',axis=1).columns.tolist(),
                   random_state=1)


### 13. Get Local Explanations for individual samples
Once LIME Tabular has been initialized, it can be used to explain the prediction for specific samples. 
The below step executes location explanations for samples with index 0 to 19. It may take more than a minute to complete. Ignore the warnings. Once complete, it will display drop down with predictions for each of the 20 samples. Selecting a particular sample in the drop down, will display the local explanations for the prediction for the paricular sample. 
Observe the predictions. 

In order to deep dive let us pick 2 examples with one where the model classifies as 'Stroke' and another where the model classifies as 'No Stroke'. 

The record in dropdown with index 0 has prediction for Stroke and may look like -  '0: Predicted (0.6) | Actual (1.0)' | Resid (0.4)'. 
The model has predicted 'Stroke' with probability around 0.6 and 'No Stroke' with probability around 0.4. Observe the individual features contributing to positive Stroke and negative Stroke. The features in the right side in Orange color are the ones that are contributing to Stroke and the features in the left side in Blue color are the ones contributing to 'No Stroke'. As you can see, the features contributing to 'Stroke' is higher than the ones contributing to 'No Stroke'

The record in dropdown with index 1 has prediction for 'No Stroke' -  '1: Predicted (0.01) | Actual (0)' | Resid (-0.01)'. 
The model has predicted 'No Stroke' with probability around 0.9x and 'Stroke' with probability around 0.0x. Observe the individual features contributing to 'No Stroke' and 'Stroke'. As you can see, the features contributing to 'No Stroke' (left side - Blue) is lot more than the ones contributing to 'Stroke' (right side - Orange)

Please ignore UserWarning, if any related to deprecated package usage

In [ ]:
# Get local explanations
lime_local = lime.explain_local(X_test[0:20], 
                                y_test[0:20], 
                                name='LIME')

show(lime_local)

### 14. Initiate LIME using lime library
The below step initializes LIME for Tabular data analysis using Lime library instead of the interpretML library. It shows different representation of the same data. You may chose either the above interpretML's visualization or the below view, whichever is more comprehensible.

In [ ]:
import lime
from lime import lime_tabular

explainer = lime_tabular.LimeTabularExplainer( training_data = np.array(X_train), feature_names=X_train.columns,class_names=['0 - No Stroke','1 - Stroke'],mode='classification')


### 15. Visualize local explanations using LIME for Classification 1 - Stroke
In the below step, local explanation is done for index 0, which is predicted as 'Stroke' candidate. The model has predicted 'Stroke' with probability around 0.6 and 'No Stroke' with probability around 0.4. Observe the individual features contributing to positive Stroke and negative Stroke. The features in the right side in Orange color are the ones that are contributing to Stroke and the features in the left side in Blue color are the ones contributing to 'No Stroke'. As you can see, the features contributing to 'Stroke' is higher than the ones contributing to 'No Stroke'

In [ ]:
record_index = 0
l=X_test.iloc[record_index, :]
exp = explainer.explain_instance(data_row=l, predict_fn = rf.predict_proba)
# print(l)
print('Actual outcode:' + str(y_test.iloc[record_index]))
exp.show_in_notebook(show_table=True)

### 16. Visualize local explanations using LIME for Classification 0 - No Stroke
In the below step, local explanation is done for index 1, which is predicted as 'No Stroke' candidate. 

The model has predicted 'No Stroke' with probability around 0.95 and 'Stroke' with probability around 0.05. Observe the individual features contributing to positive Stroke and negative Stroke. The features in the right side in Orange color are the ones that are contributing to Stroke and the features in the left side in Blue color are the ones contributing to 'No Stroke'. As you can see, the features contributing to 'No Stroke' (left side - Blue color) is higher than the ones contributing to 'Stroke' (right side - Orange color)

In [ ]:
record_index = 1
l=X_test.iloc[record_index, :]
exp = explainer.explain_instance(data_row=l, predict_fn = rf.predict_proba)
# print(l)
print('Actual outcode:' + str(y_test.iloc[record_index]))
exp.show_in_notebook(show_table=True)

# SHAP - SHapley Additive exPlanation
SHapley Additive exPlanation, acronymed SHAP, is another model agnostic technique used to explain predictions like LIME. SHAP is based on game theory concept created by Lloyd Stowell Shapley in 1950s. It was originally used to find the marginal contribution of each of the individual players in a coalition of players, so that a lumpsum of price money can be distributed among the players according to their contribution, instead of equal split. The contribution of each player is quantified as Shapley value.

Machine learning prediction is considered as a game. Features are players playing together to bring an outcome (prediction). SHAP is used to calculate the average marginal contribution of a feature value over all possible coalitions of features. The details of the mathematical model behind SHAP is explained in the research paper "A Unified Approach to Interpreting Model Predictions" by Scott Lundberg and Su-In Lee. Fortunately we do not have to implement the mathematical model ourselves. The SHAP library used below abstracts the mathematical model and simplifies the usage.





### 17. Create SHAP Explainer
The below step creates SHAP explainer using TreeExplainer. The explainer instance will be used in subsequent steps to explain SHAP values.

TreeExplainer is a fast and accurate algorithm used in all kinds of tree-based models like Random Forests. SHAP also has DeepExplainer for Deep learning models and KernelExplainer for any machine learning regression model.

In [ ]:
import shap

# %% Create SHAP explainer
explainer = shap.TreeExplainer(rf)


### 18. Select record for explainability using SHAP for Classification 1 - Stroke
In the below step, the record with index 0 is selected for explaining using SHAP. This is the same record that was earlier analyzed using LIME as well, which is predicted as 'Stroke' candidate with probability around 0.6. 

The values of the features for that record is displayed for review.

In [ ]:
# Calculate shapley values for test data
record_index = 0
end_index = record_index + 1  # to analyze 1 sample record. 
X_test[record_index:end_index]

### 19. Print Shapley values
The below step displays the Shapley values for the selected record. It gives Shapley values for each feature for either of the Classifications i.e. for predicting 'No Stroke' and 'Stroke'. Observe how for each feature the Shapley values negates between the mutually exclusive classifications.

In [ ]:
shap_values = explainer.shap_values(X_test[record_index:end_index])

# class 0 = contribution to No Stroke
# class 1 = contribution to Stroke
# creating temporary Dataframe to view Shapley value in readable table
temp_df = pd.DataFrame(shap_values[0].T, X_test.columns, ['Shapley values for No Stroke'])
temp_df['Shapley values for Stroke'] = shap_values[1].T
temp_df


### 20. Visualize the Shapley values for Classification 1 - Stroke
While the above step displayes the quantified Shapley values, it may not be very helpful in appreciating the relative comparison. In the below step, we sill visualize the Shapley values for the selected record. Observe individual features contributing to the prediction.

For this record, the overall prediction of 'Stroke' is around 0.6x. The features in the left side, in red color, contribute to 'Stroke'. The features in the right side, in blue color, contribute to 'no Stroke'. The width of each feature is its Shapley value and the total width is the cummulative Shapley values for all the attributes favoring the particular classification. In this case the Shapley values of the features contributing to Stroke is higher compared to the Shapley values of features contributing to 'no Stroke'. Hence the prediction was made as 'Stroke'.

In [ ]:
shap.initjs()
prediction = rf.predict(X_test[record_index: end_index])[0]
 
shap.force_plot(explainer.expected_value[1],
                shap_values[1],
                X_test[record_index: end_index]) # for values
 

### 21. Repeat the visualization for a different sample with Classification 0 - No Stroke

The below step visualizes for a different sample with index '1' which has prediction as '0 - No Stroke'. In order to visualize the feature contributions to Classification 0, while plotting the graph, Shapley values in array index '0' in variable 'shap_values' is taken, unlike the earlier example where the contributions to Classification 1 was visualized.

The below graph shows the prediction of 'No Stroke' with probability of around 0.95 and the features in the left side in red color contribute to it and the ones against it in Blue color. You can notice that unlike the earlier example, in this case the Age feature is contributing to 'No Stroke'. 

Try changing the record_index and end_index to different values and observe the Shapley value contributions for that particular record.

In [ ]:
record_index = 1
end_index = record_index + 1  # to analyze 1 sample record. 
shap_values = explainer.shap_values(X_test[record_index:end_index])

shap.initjs()
prediction = rf.predict(X_test[record_index:end_index])[0]

# print(f"The predicted value: {prediction}")
shap.force_plot(explainer.expected_value[1],
                shap_values[0],
                X_test[record_index:end_index]) # for values
 

### 22. Visualize Shepley values for group of records
In the below step, 20 records from index 0 to 19 are analyzed. 

The graph has drop downs in both the dimensions. By default 'sample order by similarity' is selected in the dropdown for the X Axis that shows the instance index and for dropdown in Y axis, the Shapley values (function f(x)) is defaulted. On mouse over in the graph, the particular instance's feature values and the overall Shapley values are highlighted.

Explore the visualization by changing the dropdown values. First you can try changing the value in the dropdown in Y axis to see the effect of individual feature for each of the instance. 
For example, to understand the impact of feature age, select 'age' in the dropdown in Y axis. You can see that higher the age, the chances of stroke is higher. Similarly you can try checking for other features. You can also change the dropdown in X axis and visualize the impact.


In [ ]:
start_index = 0
end_index = 20
shap_values = explainer.shap_values(X_test[start_index:end_index])
shap.initjs()
prediction = rf.predict(X_test[start_index:end_index])[0]
# print(f"The RF predicted: {prediction}")
shap.force_plot(explainer.expected_value[1],
                shap_values[1],
                X_test[start_index:end_index]) # for values
# shap.summary_plot(shap_values, X_test)

### 23. Visualize Global Features
The below step displays the summary plot for the 20 instances. As you can see the features age, avg_glucose_level, bmi play vital role in selecting Class 0 or 1.

In [ ]:
# %% >> Visualize global features
# Feature summary
shap.summary_plot(shap_values, X_test)

#### Congratulations!  You have successfully analyzed the predictions of model using LIME and SHAP explainable AI libraries 

Possible next step for learning:
* You can try changing Randow Forest Classifier to other algorithms and try the same exercise
* You can try using a different dataset and try the same exercise using LIME and SHAP
* You can try to apply LIME over multiclass textual data of newsgroup dataset by using the following link https://marcotcr.github.io/lime/tutorials/Lime%20-%20multiclass.html
* You can try to apply SHAP for Sentiment Analysis with Logistic Regression on IDMB dataset by using the following link https://slundberg.github.io/shap/notebooks/linear_explainer/Sentiment%20Analysis%20with%20Logistic%20Regression.html

Reference for above exercise: 'Learn XAI in Simplified way by Prof.Prateek Bhatia' in Udemy